In [19]:
import os
from dotenv import load_dotenv
from llama_index.llms.gemini import Gemini
from llama_index.embeddings.google_genai import GoogleGenAIEmbedding
from pinecone import Pinecone,ServerlessSpec
from llama_index.vector_stores.pinecone import PineconeVectorStore
load_dotenv(override=True)



ModuleNotFoundError: No module named 'dotenv'

In [20]:
embed_model = GoogleGenAIEmbedding(model_name="text-embedding-004")
llm = Gemini(model_name="gemini-1.5-flash",
            #  generation_config={
            #      "max_output_tokens":500
            #  })
)

NameError: name 'GoogleGenAIEmbedding' is not defined

In [21]:
os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY")
os.environ["PINECONE_API_KEY"] = "pcsk_5fdr4r_Trs2oDXqMb5fddwu2ozmLnrJ35rvGqxbZRzCjpLFmRvTXnXtwXHiJc4UDrK5CX"
api_key = os.environ["PINECONE_API_KEY"]

TypeError: str expected, not NoneType

In [22]:
pc=Pinecone(api_key=api_key)

NameError: name 'Pinecone' is not defined

In [23]:
print(os.environ["GOOGLE_API_KEY"])

KeyError: 'GOOGLE_API_KEY'

In [47]:
from llama_index.core import SimpleDirectoryReader,VectorStoreIndex
from llama_index.core import Settings
Settings.chunk_size=100
Settings.chunk_overlap=10

documents = SimpleDirectoryReader("data").load_data()

In [48]:
documents

[Document(id_='fc538db2-0fea-42cf-af54-9c5e2dcfdee3', embedding=None, metadata={'page_label': '1', 'file_name': 'MCA_III_TimeTable.pdf', 'file_path': 'c:\\Users\\ASUS\\Desktop\\LLamaindex\\RAG Production\\data\\MCA_III_TimeTable.pdf', 'file_type': 'application/pdf', 'file_size': 1921, 'creation_date': '2025-09-15', 'last_modified_date': '2025-09-13'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text_resource=MediaResource(embeddings=None, data=None, text='MCA III Time Table\nDepartment of Computer Science and Engineering Institute of Engineering and Technology Time\nTable Odd Semester 2025 26 MCA III Class Room LT 15\nOn Monday the first period from 9 10 to 10 00 AM is KCA 021 lecture by HM follo

In [43]:
# NOW CREATING EMBEDDINGS
index = VectorStoreIndex.from_documents(documents,embed_model=embed_model)

2025-09-16 18:18:36,112 - INFO - HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/text-embedding-004:batchEmbedContents "HTTP/1.1 200 OK"
2025-09-16 18:18:36,891 - INFO - HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/text-embedding-004:batchEmbedContents "HTTP/1.1 200 OK"


In [49]:
pinecone_index = pc.Index("rag-index") # we need to create an index with pinecone to store our embeddings


In [53]:

# initialize without metadata filter
from llama_index.core import StorageContext

if "GOOGLE_API_KEY" not in os.environ:
    raise EnvironmentError(f"Environment variable OPENAI_API_KEY is not set")

vector_store = PineconeVectorStore(pinecone_index=pinecone_index)
storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = VectorStoreIndex.from_documents(
    documents, storage_context=storage_context,
    embed_model=embed_model
)

2025-09-16 18:30:01,206 - INFO - HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/text-embedding-004:batchEmbedContents "HTTP/1.1 200 OK"
2025-09-16 18:30:01,975 - INFO - HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/text-embedding-004:batchEmbedContents "HTTP/1.1 200 OK"
Upserted vectors: 100%|██████████| 11/11 [00:02<00:00,  4.08it/s]


In [ ]:
# set Logging to DEBUG for more detailed outputs
query_engine1 = index.as_query_engine()
response = query_engine1.query("Who are the teachers in the timtable ?")
response

ValueError: 
******
Could not load OpenAI model. If you intended to use OpenAI, please check your OPENAI_API_KEY.
Original error:
No API key found for OpenAI.
Please set either the OPENAI_API_KEY environment variable or openai.api_key prior to initialization.
API keys can be found or created at https://platform.openai.com/account/api-keys

To disable the LLM entirely, set llm=None.
******

In [54]:
index # vectors created 

In [55]:
from llama_index.core.response_synthesizers import get_response_synthesizer
response_synthesizer = get_response_synthesizer(llm=llm)

In [56]:
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.postprocessor import SimilarityPostprocessor
postprocessor = SimilarityPostprocessor(similarity_cutoff=0.77) # for similarity greater than 
retriever = VectorIndexRetriever(index=index, similarity_top_k=2) # takes out the top chunks from the database
query_engine = RetrieverQueryEngine(retriever=retriever, response_synthesizer= response_synthesizer,node_postprocessors=[postprocessor]) # retriever engine give the chunks and text to the llm for text generation 

In [57]:
response = query_engine.query("What is TimeTable of MCA")


2025-09-16 18:32:19,970 - INFO - HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/text-embedding-004:batchEmbedContents "HTTP/1.1 200 OK"


In [58]:
from llama_index.core.response import pprint_utils
pprint_utils.pprint_response(response,show_source=True)
print(response)





Final Response: Empty Response
Empty Response


In [33]:



# print(response)